# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.1080, train_loss: 2.18634319, train_accuracy: 0.4375, test_Accuracy: 0.3118
Epoch: [ 0] [    1/  468] time: 1.2068, train_loss: 2.11115575, train_accuracy: 0.5625, test_Accuracy: 0.4380
Epoch: [ 0] [    2/  468] time: 1.3005, train_loss: 2.03562689, train_accuracy: 0.6016, test_Accuracy: 0.4966
Epoch: [ 0] [    3/  468] time: 1.4052, train_loss: 2.03823519, train_accuracy: 0.5391, test_Accuracy: 0.5530
Epoch: [ 0] [    4/  468] time: 1.4930, train_loss: 1.95295227, train_accuracy: 0.5781, test_Accuracy: 0.6003
Epoch: [ 0] [    5/  468] time: 1.5888, train_loss: 1.80353856, train_accuracy: 0.7031, test_Accuracy: 0.6299
Epoch: [ 0] [    6/  468] time: 1.6785, train_loss: 1.75341916, train_accuracy: 0.6406, test_Accuracy: 0.6446
Epoch: [ 0] [    7/  468] time: 1.7792, train_loss: 1.65401232, train_accuracy: 0.6641, test_Accuracy: 0.6506
Epoch: [ 0] [    8/  468] time: 1.8720,

Epoch: [ 0] [   73/  468] time: 8.0455, train_loss: 0.35306498, train_accuracy: 0.8906, test_Accuracy: 0.8932
Epoch: [ 0] [   74/  468] time: 8.1412, train_loss: 0.37135011, train_accuracy: 0.8750, test_Accuracy: 0.8965
Epoch: [ 0] [   75/  468] time: 8.2310, train_loss: 0.19523250, train_accuracy: 0.9297, test_Accuracy: 0.9027
Epoch: [ 0] [   76/  468] time: 8.3277, train_loss: 0.27763301, train_accuracy: 0.9375, test_Accuracy: 0.9041
Epoch: [ 0] [   77/  468] time: 8.4185, train_loss: 0.24755093, train_accuracy: 0.9297, test_Accuracy: 0.9015
Epoch: [ 0] [   78/  468] time: 8.5092, train_loss: 0.34600976, train_accuracy: 0.8750, test_Accuracy: 0.9053
Epoch: [ 0] [   79/  468] time: 8.5990, train_loss: 0.24005878, train_accuracy: 0.9453, test_Accuracy: 0.9067
Epoch: [ 0] [   80/  468] time: 8.6957, train_loss: 0.32693118, train_accuracy: 0.9141, test_Accuracy: 0.9103
Epoch: [ 0] [   81/  468] time: 8.7845, train_loss: 0.35546857, train_accuracy: 0.8984, test_Accuracy: 0.9139
Epoch: [ 0

Epoch: [ 0] [  148/  468] time: 15.0268, train_loss: 0.21428855, train_accuracy: 0.9297, test_Accuracy: 0.9188
Epoch: [ 0] [  149/  468] time: 15.1395, train_loss: 0.27787650, train_accuracy: 0.9219, test_Accuracy: 0.9252
Epoch: [ 0] [  150/  468] time: 15.2402, train_loss: 0.24429613, train_accuracy: 0.9297, test_Accuracy: 0.9304
Epoch: [ 0] [  151/  468] time: 15.3290, train_loss: 0.29512709, train_accuracy: 0.9219, test_Accuracy: 0.9325
Epoch: [ 0] [  152/  468] time: 15.4198, train_loss: 0.20357569, train_accuracy: 0.9531, test_Accuracy: 0.9338
Epoch: [ 0] [  153/  468] time: 15.5185, train_loss: 0.23519652, train_accuracy: 0.9297, test_Accuracy: 0.9318
Epoch: [ 0] [  154/  468] time: 15.6092, train_loss: 0.24084970, train_accuracy: 0.9453, test_Accuracy: 0.9301
Epoch: [ 0] [  155/  468] time: 15.7000, train_loss: 0.26551920, train_accuracy: 0.8984, test_Accuracy: 0.9302
Epoch: [ 0] [  156/  468] time: 15.8007, train_loss: 0.16560951, train_accuracy: 0.9453, test_Accuracy: 0.9291
E

Epoch: [ 0] [  222/  468] time: 21.9862, train_loss: 0.14770129, train_accuracy: 0.9766, test_Accuracy: 0.9375
Epoch: [ 0] [  223/  468] time: 22.0829, train_loss: 0.15964259, train_accuracy: 0.9609, test_Accuracy: 0.9365
Epoch: [ 0] [  224/  468] time: 22.1757, train_loss: 0.13589758, train_accuracy: 0.9688, test_Accuracy: 0.9381
Epoch: [ 0] [  225/  468] time: 22.2714, train_loss: 0.22157961, train_accuracy: 0.9297, test_Accuracy: 0.9409
Epoch: [ 0] [  226/  468] time: 22.3612, train_loss: 0.16789612, train_accuracy: 0.9453, test_Accuracy: 0.9430
Epoch: [ 0] [  227/  468] time: 22.4579, train_loss: 0.11141416, train_accuracy: 0.9688, test_Accuracy: 0.9425
Epoch: [ 0] [  228/  468] time: 22.5487, train_loss: 0.19225821, train_accuracy: 0.9297, test_Accuracy: 0.9397
Epoch: [ 0] [  229/  468] time: 22.6384, train_loss: 0.19870700, train_accuracy: 0.9609, test_Accuracy: 0.9361
Epoch: [ 0] [  230/  468] time: 22.7322, train_loss: 0.21723767, train_accuracy: 0.9453, test_Accuracy: 0.9343
E

Epoch: [ 0] [  295/  468] time: 28.7700, train_loss: 0.18844877, train_accuracy: 0.9531, test_Accuracy: 0.9480
Epoch: [ 0] [  296/  468] time: 28.8678, train_loss: 0.21120790, train_accuracy: 0.9219, test_Accuracy: 0.9487
Epoch: [ 0] [  297/  468] time: 28.9685, train_loss: 0.13439815, train_accuracy: 0.9531, test_Accuracy: 0.9500
Epoch: [ 0] [  298/  468] time: 29.0593, train_loss: 0.25223160, train_accuracy: 0.9375, test_Accuracy: 0.9519
Epoch: [ 0] [  299/  468] time: 29.1500, train_loss: 0.17128967, train_accuracy: 0.9766, test_Accuracy: 0.9524
Epoch: [ 0] [  300/  468] time: 29.2458, train_loss: 0.16988248, train_accuracy: 0.9531, test_Accuracy: 0.9516
Epoch: [ 0] [  301/  468] time: 29.3375, train_loss: 0.14095615, train_accuracy: 0.9688, test_Accuracy: 0.9498
Epoch: [ 0] [  302/  468] time: 29.4293, train_loss: 0.16117337, train_accuracy: 0.9766, test_Accuracy: 0.9488
Epoch: [ 0] [  303/  468] time: 29.5190, train_loss: 0.25901479, train_accuracy: 0.9375, test_Accuracy: 0.9477
E

Epoch: [ 0] [  369/  468] time: 35.6477, train_loss: 0.06994383, train_accuracy: 0.9844, test_Accuracy: 0.9515
Epoch: [ 0] [  370/  468] time: 35.7424, train_loss: 0.16423221, train_accuracy: 0.9453, test_Accuracy: 0.9522
Epoch: [ 0] [  371/  468] time: 35.8322, train_loss: 0.17987835, train_accuracy: 0.9531, test_Accuracy: 0.9541
Epoch: [ 0] [  372/  468] time: 35.9299, train_loss: 0.09595641, train_accuracy: 0.9844, test_Accuracy: 0.9564
Epoch: [ 0] [  373/  468] time: 36.0286, train_loss: 0.18873084, train_accuracy: 0.9297, test_Accuracy: 0.9576
Epoch: [ 0] [  374/  468] time: 36.1184, train_loss: 0.15462543, train_accuracy: 0.9609, test_Accuracy: 0.9576
Epoch: [ 0] [  375/  468] time: 36.2082, train_loss: 0.15144393, train_accuracy: 0.9531, test_Accuracy: 0.9551
Epoch: [ 0] [  376/  468] time: 36.2989, train_loss: 0.15119438, train_accuracy: 0.9531, test_Accuracy: 0.9510
Epoch: [ 0] [  377/  468] time: 36.3937, train_loss: 0.12443941, train_accuracy: 0.9766, test_Accuracy: 0.9482
E

Epoch: [ 0] [  443/  468] time: 42.5173, train_loss: 0.14226022, train_accuracy: 0.9453, test_Accuracy: 0.9604
Epoch: [ 0] [  444/  468] time: 42.6090, train_loss: 0.08787997, train_accuracy: 0.9688, test_Accuracy: 0.9555
Epoch: [ 0] [  445/  468] time: 42.6978, train_loss: 0.12726405, train_accuracy: 0.9375, test_Accuracy: 0.9529
Epoch: [ 0] [  446/  468] time: 42.7905, train_loss: 0.09517679, train_accuracy: 0.9688, test_Accuracy: 0.9526
Epoch: [ 0] [  447/  468] time: 42.8843, train_loss: 0.20261770, train_accuracy: 0.9375, test_Accuracy: 0.9523
Epoch: [ 0] [  448/  468] time: 42.9771, train_loss: 0.08600446, train_accuracy: 0.9766, test_Accuracy: 0.9539
Epoch: [ 0] [  449/  468] time: 43.0678, train_loss: 0.11722119, train_accuracy: 0.9766, test_Accuracy: 0.9578
Epoch: [ 0] [  450/  468] time: 43.1705, train_loss: 0.23034823, train_accuracy: 0.9375, test_Accuracy: 0.9588
Epoch: [ 0] [  451/  468] time: 43.2673, train_loss: 0.10036685, train_accuracy: 0.9766, test_Accuracy: 0.9589
E

# Test accuracy : 96.22 %